## running elastic

check if is there any unused container: `docker system prune -af --volumes`

before importing ElasticSearch, run the server:

```
docker run -it \
    --rm \
    --name elasticsearch \
    -m 4GB \
    -p 9200:9200 \
    -p 9300:9300 \
    -e "discovery.type=single-node" \
    -e "xpack.security.enabled=false" \
    docker.elastic.co/elasticsearch/elasticsearch:8.17.6
```

In [12]:
!curl localhost:9200

{
  "name" : "9b00f8f1e732",
  "cluster_name" : "docker-cluster",
  "cluster_uuid" : "li9JE_S9R8CXrQpWml7hiw",
  "version" : {
    "number" : "8.17.6",
    "build_flavor" : "default",
    "build_type" : "docker",
    "build_hash" : "dbcbbbd0bc4924cfeb28929dc05d82d662c527b7",
    "build_date" : "2025-04-30T14:07:12.231372970Z",
    "build_snapshot" : false,
    "lucene_version" : "9.12.0",
    "minimum_wire_compatibility_version" : "7.17.0",
    "minimum_index_compatibility_version" : "7.0.0"
  },
  "tagline" : "You Know, for Search"
}


## getting the data

In [13]:
import requests 

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [14]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

## indexing the data

In [15]:
from elasticsearch import Elasticsearch
from tqdm.auto import tqdm

In [16]:
es_client = Elasticsearch('http://localhost:9200', request_timeout=60)

index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questions"

es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [17]:
import time

# Wait for the index to be ready
for _ in tqdm(range(30), desc="Waiting for index to become ready"):
    health = es_client.cluster.health(index=index_name)
    if health["status"] in ("yellow", "green"):
        break
    time.sleep(1)
else:
    raise RuntimeError("Index is not ready after waiting.")

Waiting for index to become ready:   0%|          | 0/30 [00:00<?, ?it/s]


In [18]:
for document in tqdm(documents):
    es_client.index(index=index_name, document=document) # index

100%|██████████| 948/948 [00:06<00:00, 139.81it/s]


## searching

In [37]:
def elastic_search(query: str, 
                   size: int = 5, 
                   fields: list = ["question^4", "text"], 
                   course_name: str = "data-engineering-zoomcamp"):
    search_query = {
        "size": size,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": fields,
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": course_name
                    }
                }
            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)
    
    # result_docs = []
    
    # for hit in response['hits']['hits']:
    #     result_docs.append(hit['_source'])
    
    # return result_docs

    return response

In [38]:
query = "How do execute a command on a Kubernetes pod?"

response = elastic_search(query)

In [39]:
type(response["hits"])

dict

In [40]:
response["hits"].keys()

dict_keys(['total', 'max_score', 'hits'])

In [41]:
response["hits"]["max_score"]

31.973522

In [42]:
response["hits"]["total"]

{'value': 334, 'relation': 'eq'}

In [43]:
response["hits"]["hits"]

[{'_index': 'course-questions',
  '_id': '-3Mne5cB4jH3O_jqzOD3',
  '_score': 31.973522,
  '_source': {'text': 'Install the astronomer-cosmos package as a dependency. (see Terraform example).\nMake a new folder, dbt/, inside the dags/ folder of your Composer GCP bucket and copy paste your dbt-core project there. (see example)\nEnsure your profiles.yml is configured to authenticate with a service account key. (see BigQuery example)\nCreate a new DAG using the DbtTaskGroup class and a ProfileConfig specifying a profiles_yml_filepath that points to the location of your JSON key file. (see example)\nYour dbt lineage graph should now appear as tasks inside a task group like this:',
   'section': 'Course Management Form for Homeworks',
   'question': 'How to run a dbt-core project as an Airflow Task Group on Google Cloud Composer using a service account JSON key',
   'course': 'data-engineering-zoomcamp'}},
 {'_index': 'course-questions',
  '_id': 'ZnMne5cB4jH3O_jqvd-C',
  '_score': 31.098085

## filtering

In [48]:
query = "How do copy a file to a Docker container?"

response = elastic_search(query, size=3, course_name="machine-learning-zoomcamp")

In [ ]:
response["hits"]["hits"] # How do I copy files from a different folder into docker container’s working directory?

[{'_index': 'course-questions',
  '_id': 'vXMne5cB4jH3O_jq0eEw',
  '_score': 73.38676,
  '_source': {'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)',
   'section': '5. Deploying Machine Learning Models',
   'question': 'How do I debug a docker container?',
   'course': 'machine-learning-zoomcamp'}},
 {'_index': 'course-questions',
  '_id': '3HMne5cB4jH3O_jq0eHC',
  '_score': 66.688705,
  '_source': {'text': "You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:\nTo copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:\ndocker cp /path/to/local/file_

## building a prompt

In [52]:
results = [value["_source"] for value in response["hits"]["hits"]]

In [53]:
results

[{'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)',
  'section': '5. Deploying Machine Learning Models',
  'question': 'How do I debug a docker container?',
  'course': 'machine-learning-zoomcamp'},
 {'text': "You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:\nTo copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:\ndocker cp /path/to/local/file_or_directory container_id:/path/in/container\nHrithik Kumar Advani",
  'section': '5. Deploying Machine Learning Models',
  'question': 'How do I copy files from my local machine to docker container?',
 

In [56]:
def build_prompt(question, results):
  
  prompt_template = """
    You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
    Use only the facts from the CONTEXT when answering the QUESTION.

    QUESTION: {question}

    CONTEXT:
    {context}
  """.strip()

  context = "" 

  for doc in results:
      context = context + f"\nQ: {doc['question']}\n\n" + \
          f"A: {doc['text']}\n\n"
  prompt = prompt_template.format(question=question, context=context).strip()

  return prompt

In [57]:
prompt = build_prompt(
    question="How do copy a file to a Docker container?",
    results=results
)
print(prompt)

You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
    Use only the facts from the CONTEXT when answering the QUESTION.

    QUESTION: How do copy a file to a Docker container?

    CONTEXT:
    
Q: How do I debug a docker container?

A: Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.
docker run -it --entrypoint bash <image>
If the container is already running, execute a command in the specific container:
docker ps (find the container-id)
docker exec -it <container-id> bash
(Marcos MJD)


Q: How do I copy files from my local machine to docker container?

A: You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:
To copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:
docker cp /path/to/local/file_or_directory container

In [ ]:
len(prompt) # 1446

1468

## tokens

In [60]:
# !pip install tiktoken

In [62]:
import tiktoken

In [63]:
encoding = tiktoken.encoding_for_model("gpt-4o")

In [ ]:
len(encoding.encode(prompt)) # 320 tokens

324

In [68]:
prompt[:24]

"You're a course teaching"

In [69]:
encoding.encode(prompt[:24])

[63842, 261, 4165, 14029]

## generating the answer

In [70]:
from ollama import chat
from ollama import ChatResponse

In [71]:
answer: ChatResponse = chat(model="llama3.2", messages=[
{
    "role": "user",
    "content": prompt,
},
])

final_answer = answer.message.content

In [72]:
print(final_answer)

To copy a file from your local machine into a Docker container, you can use the `docker cp` command.

Basic syntax: 
```
docker cp /path/to/local/file_or_directory container_id:/path/in/container
```
